In [1]:
#import healpy as hp
import astropy_healpix.healpy as ahp
from astropy_healpix import HEALPix
from hpmoc.healpy import healpy as hhp
from matplotlib import pyplot as plt, cm
import numpy as np
import hpmoc
from hpmoc.partial import PartialUniqSkymap
from hpmoc import atlas as atls
from reproject import reproject_from_healpix, reproject_to_healpix
import astropy as ap
from astropy.wcs import WCS
from astropy.io import fits
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.coordinates import FK5

In [2]:
m = PartialUniqSkymap.read("../tests/data/S191216ap.fits.gz", strategy='ligo')
mo = PartialUniqSkymap.read("../tests/data/S191216ap.multiorder.fits", strategy='ligo')
mc = (m.s⃗*m.A⃗()).value

E:\miniconda3\envs\hpmoc-dev-win\lib\site-packages\hpmoc\utils.py:1443: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  u⃗ = np.array(u⃗, dtype=dtype, copy=False)


In [3]:
hdu_ligo = fits.open("../tests/data/S191216ap.fits.gz")[1]
target_header = fits.Header.fromstring("""
NAXIS   =                    2
NAXIS1  =                  480
NAXIS2  =                  240
CTYPE1  = 'RA---MOL'
CRPIX1  =                240.5
CRVAL1  =                180.0
CDELT1  =               -0.675
CUNIT1  = 'deg     '
CTYPE2  = 'DEC--MOL'
CRPIX2  =                120.5
CRVAL2  =                  0.0
CDELT2  =                0.675
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')
w = WCS(target_header)

In [4]:
mo.render(w)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [5]:
# Go counter-clockwise from x=0, i.e. x=0, y=nˢ-1, x=nˢ-1, y=0
atlas = np.array([
    [4, 3, 1, 5],
    [5, 0, 2, 6],
    [6, 1, 3, 7],
    [7, 2, 0, 4],
    [11, 3, 0, 8],
    [8, 0, 1, 9],
    [9, 1, 2, 10],
    [10, 2, 3, 11],
    [11, 4, 5, 9],
    [8, 5, 6, 10],
    [9, 6, 7, 11],
    [10, 7, 4, 8],
], dtype=np.uint8)
# Go counter-clockwise from (0,0), i.e. (0,0), (0,1), (1,1), (1,0)

In [92]:
right, face = np.meshgrid(*map(np.arange, atlas.shape[::-1]))
loops = np.stack([*map(np.ravel, [np.roll(right, 1), face, right])]).T

In [180]:
def close_loops(loops):
    next_sides = loops[:,-1]
    last_faces = loops[:,-2]
    next_faces = atlas[last_faces, next_sides]
    next_last_sides = np.where(atlas[next_faces]==last_faces.reshape((-1, 1)))[1]
    is_done = (loops[:,0]==next_last_sides) & (loops[:,1]==next_faces)
    done = loops[is_done]
    
    # dedup and clean up
    k = done.shape[1]
    roll = 3*done[:,np.arange(1, k, 3)].argsort()[:,0]
    rows, col_idx = np.ogrid[:done.shape[0], :k]
    col_idx = (col_idx + roll[:, None])%k
    next_res = done[rows, col_idx]
    sorted_res = next_res[np.lexsort([next_res[:,i] for i in range(k-2, 0, -3)])]
    for i in range(1, k//3):
        assert (sorted_res[::k//3]==sorted_res[i::k//3]).all()
    final = sorted_res[::k//3].reshape((-1, k//3, 3))

    remove = (loops[is_done,:2].reshape((1, -1, 2))==loops[:,:2].reshape((-1, 1, 2))).all(axis=2).any(axis=1)
    if remove.all():
        return [final]
    next_next_sides = ((next_last_sides[~remove].repeat(2).reshape((-1, 2))+(1+2*np.arange(2)))%4).ravel()
    next_loops = np.concatenate(
        [
            loops[~remove].repeat(2, 0),
            np.stack([
                next_last_sides[~remove].repeat(2),
                next_faces[~remove].repeat(2),
                next_next_sides
            ]).T
        ],
        1,
    )
    return [final, *close_loops(next_loops)]

In [181]:
close_loops(loops)

[array([], shape=(0, 1, 3), dtype=int32),
 array([], shape=(0, 2, 3), dtype=int64),
 array([[[ 0,  0,  1],
         [ 2,  3,  3],
         [ 1,  4,  2]],
 
        [[ 2,  0,  3],
         [ 1,  5,  2],
         [ 0,  1,  1]],
 
        [[ 2,  1,  3],
         [ 1,  6,  2],
         [ 0,  2,  1]],
 
        [[ 2,  2,  3],
         [ 1,  7,  2],
         [ 0,  3,  1]],
 
        [[ 3,  4,  0],
         [ 2, 11,  3],
         [ 0,  8,  1]],
 
        [[ 3,  5,  0],
         [ 2,  8,  3],
         [ 0,  9,  1]],
 
        [[ 3,  6,  0],
         [ 2,  9,  3],
         [ 0, 10,  1]],
 
        [[ 3,  7,  0],
         [ 2, 10,  3],
         [ 0, 11,  1]]], dtype=int64),
 array([[[ 1,  0,  2],
         [ 1,  1,  2],
         [ 1,  2,  2],
         [ 1,  3,  2]],
 
        [[ 3,  0,  0],
         [ 2,  4,  3],
         [ 1,  8,  2],
         [ 0,  5,  1]],
 
        [[ 3,  1,  0],
         [ 2,  5,  3],
         [ 1,  9,  2],
         [ 0,  6,  1]],
 
        [[ 3,  2,  0],
         [ 2,  6,  

In [9]:
f'{0x5555555555555555:064b}'

'0101010101010101010101010101010101010101010101010101010101010101'

In [12]:
f'{0b1111:x}'

'f'

In [26]:
from hpmoc.utils import uniq_intersection, uniq2nest_and_nside, uniq2nest, uniq2nside

uniq_intersection(np.array([3003, 301]), np.arange(4, 16))

E:\miniconda3\envs\hpmoc-dev-win\lib\site-packages\hpmoc\utils.py:1443: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  u⃗ = np.array(u⃗, dtype=dtype, copy=False)


(array([0, 1], dtype=int64), array([7, 0], dtype=int64), array([-4, -3]))

In [35]:
x = np.array([3003, 301])
(x//uniq2nside(x)**2)-4

array([7, 0], dtype=int32)

In [31]:
uniq2nest_and_nside(x)

(array([1979,   45]), array([16,  8], dtype=int32))

In [7]:
hp = ahp

In [11]:
hp.ring2nest(2, 4)

array([2], dtype=int64)

In [23]:
np.isscalar(hp.ring2nest(1, 8))

False

In [35]:
hp.order2nside(3) == 1<<3

True

In [50]:
np.random.randint?

In [52]:
type(np.random.randint(0, 30))

int

In [49]:
np.random.randint(0, 30, 10, dtype='uint64')

array([28, 15, 23, 27,  7,  6,  9, 11,  3, 10], dtype=uint64)

In [38]:
def pix2xyf(nside, ipix, nest=False):
    "Drop-in replacement for ``healpy.pix2xyf``."
    import numpy as np

    # Check for mistake in ``astropy_healpix`` scalar handling
    scalar = np.isscalar(ipix)
    ipix = ipix if nest else hp.ring2nest(nside, ipix)
    if scalar and not np.isscalar(ipix):
        ipix = ipix.ravel()[0]
    nsq = nside*nside
    f = ipix//nsq
    i = ipix-f*nsq
    return alt_compress(i), alt_compress(i>>1, True), f


def xyf2pix(nside, x, y, face, nest=False):
    "Drop-in replacement for ``healpy.xyf2pix``."
    import numpy as np

    # Check for mistake in ``astropy_healpix`` scalar handling
    scalar = all(map(np.isscalar, [x, y, face]))
    ipix = alt_expand(x) + (alt_expand(y)<<1) + face*nside*nside
    ipix = ipix if nest else hp.nest2ring(nside, ipix)
    return ipix.ravel()[0] if scalar and not np.isscalar(ipix) else ipix

In [44]:
xyf2pix(8, *pix2xyf(8, np.array([258, 259])))

array([258, 259], dtype=int64)

In [40]:
pix2xyf(8, xyf2pix(8, 0, 3, 2))

(0, 3, 2)

In [33]:
pix2xyf(2, 20, nest=True)

(0, 0, 5)

In [73]:
def uniq2xyf_nside(u⃗):
    """
    Examples
    --------
    >>> uniq2xyf_nside(8)
    (0, 0, 4, 1)
    
    See Also
    --------
    xyf_nside2uniq
    """
    i, nˢ = uniq2nest_and_nside(u⃗)
    nˢˢ = nˢ*nˢ
    f = (u⃗//nˢˢ)-4
    i -= f*nˢˢ
    return alt_compress(i), alt_compress(i>>1, True), f, nˢ


def xyf_nside2uniq(x, y, f, nˢ):
    """
    Examples
    --------
    >>> (xyf_nside2uniq(*uniq2xyf_nside(np.arange(4, 10000))) == np.arange(4, 10000)).all()
    True
    
    See Also
    --------
    uniq2xyf_nside
    """
    return alt_expand(x) + (alt_expand(y)<<1) + (f+4)*nˢ*nˢ

In [82]:
(xyf_nside2uniq(*uniq2xyf_nside(np.arange(4, 10000))) == np.arange(4, 10000)).all()

True

In [75]:
uniq2xyf_nside(8)

(0, 0, 4, 1)

In [76]:
xyf_nside2uniq(0, 0, 4, 1)

8

In [72]:
uniq2xyf(np.arange(4, 32))

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1], dtype=int32),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  0,  0,  0,  1,
         1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3], dtype=int32))

In [18]:
COMPRESS_MASKS = [
    0x5555555555555555,
    0x3333333333333333,
    0x0f0f0f0f0f0f0f0f,
    0x00ff00ff00ff00ff,
    0x0000ffff0000ffff,
    0x00000000ffffffff,
]


def alt_compress(x, in_place=False):
    """
    Start in 0x55... state.
    
    https://help.dyalog.com/18.0/Content/Language/Primitive%20Functions/Replicate.htm
    
    Examples
    --------
    >>> alt_compress(0b011101)
    7
    >>> alt_compress(0b110010)
    4
    >>> alt_compress(100)
    10
    >>> f'{alt_compress(0b10011100):04b}'
    '0110'
    """
    import numpy as np

    if (not isinstance(x, int)) and (not in_place):
        x = x.copy()
    x &= COMPRESS_MASKS[0]
    for i, m in enumerate(COMPRESS_MASKS[1:]):
        hold = m&x
        x &= ~m
        x >>= 1<<i
        x |= hold
    return x


def alt_expand(x, in_place=False):
    """
    Start in 0x00000000ffffffff state.
    
    https://help.dyalog.com/18.0/Content/Language/Primitive%20Functions/Expand.htm
    
    Examples
    --------
    >>> f'{alt_expand(0b100101):012b}'
    '010000010001'
    """
    import numpy as np
    
    o = len(COMPRESS_MASKS)
    if (not isinstance(x, int)) and (not in_place):
        x = x.copy()
    x &= COMPRESS_MASKS[-1]
    for i, m in enumerate(COMPRESS_MASKS[-2::-1]):
        hold = m&x
        x &= ~m
        x <<= 1<<(o-i-2)
        x |= hold
    return x
